Topic Modeling with LDA
-----------------------

### Notebook description

[Flashback](https://www.flashback.org/) and
[Familjeliv](https://www.familjeliv.se/forum) are two well-known swedish
online forums. Flashback is one of Sweden's most visited websites and
has discussions on a wide-range of topics. It is famous for emphasizing
freedom of speech and the members citizen journalism on many topics. The
online forum Familjeliv, on the other hand, focuses more on questions
about pregnancy, children and parenthood (the translation is "Family
Life"). Part of the forum is also for more general topics, and this part
of Familjeliv is probably more famous than its other parts.

What we want to do in this notebook is analyze the language used in
these two, quite different, online forums. An interesting approach we
will try here is to do some topic modeling with [Latent Dirichlet
Allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)
(LDA). We know that each discussion forum in our data has multiple
subforums, so it would be interesting to see if LDA can accurately pick
up these different subforums as different topics, or if it sees other
patterns in the threads (the threads are what corresponds to documents
in the LDA method). We will also mainly do this with forums that have a
correspondence in both Flashback and Familjeliv. For instance, both have
forums dedicated to questions regarding economy and also forums
discussing topics close to sex and relationships.

This notebook is to a large extent adapted from notebook
034*LDA*20NewsGroupsSmall.

### Downloading the data

Data comes from [språkbanken](https://spraakbanken.gu.se), see notebook
00*download*data.

In [ ]:
/scalable-data-science/000_0-sds-3-x-projects/student-project-01_group-TheTwoCultures/00_download_data

  

### Preprocessing and loading the relevant data

In [ ]:
// Here we just import methods from notebook 01_load_data, so we can load our data

%run /scalable-data-science/000_0-sds-3-x-projects/student-project-01_group-TheTwoCultures/01_load_data

  

  

  

>     import org.apache.spark.sql.functions.{col, concat_ws, udf, flatten, explode, collect_list, collect_set, lit}
>     import org.apache.spark.sql.types.{ArrayType, StructType, StructField, StringType, IntegerType}
>     import com.databricks.spark.xml._
>     import org.apache.spark.sql.functions._
>     read_xml: (file_name: String)org.apache.spark.sql.DataFrame
>     get_dataset: (file_name: String)org.apache.spark.sql.DataFrame

  

[TABLE]

  

>     save_df: (df: org.apache.spark.sql.DataFrame, filePath: String)Unit
>     load_df: (filePath: String)org.apache.spark.sql.DataFrame
>     no_forums: (df: org.apache.spark.sql.DataFrame)Long

  

  

>     familjeliv-allmanna-ekonomi.xml already exists!
>     familjeliv-sexsamlevnad.xml already exists!
>     flashback-ekonomi.xml already exists!
>     flashback-sex.xml already exists!
>     import sys.process._
>     fl_data: Array[String] = Array(familjeliv-allmanna-ekonomi.xml, familjeliv-sexsamlevnad.xml)
>     fb_data: Array[String] = Array(flashback-ekonomi.xml, flashback-sex.xml)
>     url: String = http://spraakbanken.gu.se/lb/resurser/meningsmangder/
>     tmp_folder_fl: String = /tmp/familjeliv/
>     tmp_folder_fb: String = /tmp/flashback/
>     res3: String = ""

  

>     dbfs:/datasets/student-project-01/flashback/familjeliv-allmanna-ekonomi_df
>     familjeliv-allmanna-ekonomi_df already exists!
>     dbfs:/datasets/student-project-01/flashback/familjeliv-sexsamlevnad_df
>     familjeliv-sexsamlevnad_df already exists!
>     dbfs:/datasets/student-project-01/flashback/flashback-ekonomi_df
>     flashback-ekonomi_df already exists!
>     dbfs:/datasets/student-project-01/flashback/flashback-sex_df
>     flashback-sex_df already exists!
>     fl_root: String = dbfs:/datasets/student-project-01/familjeliv/
>     fb_root: String = dbfs:/datasets/student-project-01/flashback/
>     fl_data: Array[String] = Array(familjeliv-allmanna-ekonomi, familjeliv-sexsamlevnad)
>     fb_data: Array[String] = Array(flashback-ekonomi, flashback-sex)

  

### LDA Methods

Below we put some of the LDA pipeline, working with
[MLlib](https://spark.apache.org/docs/latest/ml-guide.html), into a
single function called get\_lda.

In [ ]:
import org.apache.spark.ml.feature.{RegexTokenizer, StopWordsRemover, CountVectorizer}
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
import org.apache.spark.sql.types.{ LongType }
import org.apache.spark.ml.linalg.Vector

def get_lda(df: org.apache.spark.sql.DataFrame,  // Main function to do LDA
            min_token: Int,
            vocabSize: Int,
            minDF: Int,            
            numTopics: Int,
            maxIter: Int,
            stopWords: Array[String]) = { 

  val corpus_df = df.select("w", "thread_id").withColumn("thread_id",col("thread_id").cast(LongType)) // From the whole dataframe we take out each thread

  val tokenizer = new RegexTokenizer()
  .setPattern("(?U)[\\W_]+") // break by white space character(s)  - try to remove emails and other patterns
  .setMinTokenLength(min_token) // Filter away tokens with length < min_token
  .setInputCol("w") // name of the input column
  .setOutputCol("tokens") // name of the output column
  
  val remover = new StopWordsRemover() 
  .setStopWords(stopWords)
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("filtered")
  
  val tokenized_df = tokenizer.transform(corpus_df) // Removes uninteresting words from each thread
  val filtered_df = remover.transform(tokenized_df).select("thread_id","filtered") // Chosing only the filtered threads 
  
  val vectorizer = new CountVectorizer() // Creates dictionary and counts the occurences of different words
   .setInputCol(remover.getOutputCol)
   .setOutputCol("features")
   .setVocabSize(vocabSize) // Size of dictonary
   .setMinDF(minDF)
   .fit(filtered_df) // returns CountVectorizerModel
  
  val lda = new LDA() // Creates LDA Model with some user defined choice of parameters
  .setOptimizer(new OnlineLDAOptimizer().setMiniBatchFraction(0.8))
  .setK(numTopics)
  .setMaxIterations(maxIter)
  .setDocConcentration(-1) // use default values
  .setTopicConcentration(-1)
  
  val countVectors = vectorizer.transform(filtered_df).select("thread_id", "features")

  val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }
  val lda_countVector_mllib = lda_countVector.map { case (id, vector) => (id, org.apache.spark.mllib.linalg.Vectors.fromML(vector)) }.rdd
  
  val lda_model = lda.run(lda_countVector_mllib)
  (lda_model, vectorizer)
}

  

>     import org.apache.spark.ml.feature.{RegexTokenizer, StopWordsRemover, CountVectorizer}
>     import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
>     import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
>     import org.apache.spark.sql.types.LongType
>     import org.apache.spark.ml.linalg.Vector
>     get_lda: (df: org.apache.spark.sql.DataFrame, min_token: Int, vocabSize: Int, minDF: Int, numTopics: Int, maxIter: Int, stopWords: Array[String])(org.apache.spark.mllib.clustering.LDAModel, org.apache.spark.ml.feature.CountVectorizerModel)

  

#### Stopwords

Stop words are highly relevant to get interesting topics distributions,
with not all weight on very common words that do not carry much meaning
for a particular topic. To do this we both use collections by others and
input words we find meaningless for these particular settings.

In [ ]:
wget https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv -O /tmp/stopwords.csv


  

>     --2021-01-11 14:30:09--  https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv
>     Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
>     Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1936 (1.9K) [text/plain]
>     Saving to: ‘/tmp/stopwords.csv’
>
>          0K .                                                     100% 11.5M=0s
>
>     2021-01-11 14:30:09 (11.5 MB/s) - ‘/tmp/stopwords.csv’ saved [1936/1936]

In [ ]:
cp file:/tmp/stopwords.csv dbfs:/datasets/student-project-01/stopwords.csv

  

>     res24: Boolean = true

In [ ]:
// Creating a list with stopwords

import org.apache.spark.ml.feature.StopWordsRemover
val stoppord = sc.textFile("dbfs:/datasets/student-project-01/stopwords.csv").collect()
val stopwordList = Array("bara","lite","finns","vill","samt","inga","även","finns","ganska","också","igen","just","that","with","http","jpg","kanske","tycker","gillar", "bra","the","000","måste","tjej","tjejer","tjejen","tjejerna","kvinna","kvinnor","kille","killar","killen","män","rätt","män","com","and","html","många","aldrig","www","mpg","avi","wmv","fan","förhelvetet","riktigt","känner","väldigt","font","size","mms","2008","2009","95kr","dom","får","ska","kommer","två","vet","mer","pengar","pengarna","göra","fick","tror","andra","helt","kunna","behöver","betala","inget","dock","inget","tack"
).union(stoppord).union(StopWordsRemover.loadDefaultStopWords("swedish")) // In this step we add custom stop words and another premade list of stopwords.

  

>     import org.apache.spark.ml.feature.StopWordsRemover
>     stoppord: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)
>     stopwordList: Array[String] = Array(bara, lite, finns, vill, samt, inga, även, finns, ganska, också, igen, just, that, with, http, jpg, kanske, tycker, gillar, bra, the, 000, måste, tjej, tjejer, tjejen, tjejerna, kvinna, kvinnor, kille, killar, killen, män, rätt, män, com, and, html, många, aldrig, www, mpg, avi, wmv, fan, förhelvetet, riktigt, känner, väldigt, font, size, mms, 2008, 2009, 95kr, dom, får, ska, kommer, två, vet, mer, pengar, pengarna, göra, fick, tror, andra, helt, kunna, behöver, betala, inget, dock, inget, tack, word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre, och, det, att, i, en, jag, hon, som, han, på, den, med, var, sig, för, så, till, är, men, ett, om, hade, de, av, icke, mig, du, henne, då, sin, nu, har, inte, hans, honom, skulle, hennes, där, min, man, ej, vid, kunde, något, från, ut, när, efter, upp, vi, dem, vara, vad, över, än, dig, kan, sina, här, ha, mot, alla, under, någon, eller, allt, mycket, sedan, ju, denna, själv, detta, åt, utan, varit, hur, ingen, mitt, ni, bli, blev, oss, din, dessa, några, deras, blir, mina, samma, vilken, er, sådan, vår, blivit, dess, inom, mellan, sådant, varför, varje, vilka, ditt, vem, vilket, sitta, sådana, vart, dina, vars, vårt, våra, ert, era, vilkas)

  

### Experiments

Below we we chosen some similar forums from FB and FL. First two
regarding economics, and two discussing sex and relationships. Here we
run the two economics forums and see how well TDA captures any topic
structure.

Here one does not have to use similar forums, but if one is interested
in only Flashback forums one could use only those. We could also of
course join several of these forums together to try to capture even
broader topic distributions

In [ ]:
// Loading the forums we will do LDA on.

val file_path_FL = "dbfs:/datasets/student-project-01/familjeliv/familjeliv-allmanna-ekonomi_df"
val file_path_FB = "dbfs:/datasets/student-project-01/flashback/flashback-ekonomi_df"
//val file_path_FL = "dbfs:/datasets/student-project-01/familjeliv/familjeliv-sexsamlevnad_df"
//val file_path_FB = "dbfs:/datasets/student-project-01/flashback/flashback-sex_df"
  
val df_FL = load_df(file_path_FL)
val df_FB = load_df(file_path_FB)

  

>     file_path_FL: String = dbfs:/datasets/student-project-01/familjeliv/familjeliv-allmanna-ekonomi_df
>     file_path_FB: String = dbfs:/datasets/student-project-01/flashback/flashback-ekonomi_df
>     df_FL: org.apache.spark.sql.DataFrame = [thread_id: string, thread_title: string ... 5 more fields]
>     df_FB: org.apache.spark.sql.DataFrame = [thread_id: string, thread_title: string ... 5 more fields]

In [ ]:
// Dataframes containg a forum from FB.

df_FB.show

  

>     +---------+--------------------+--------------------+--------+--------------------+---------+--------------------+
>     |thread_id|        thread_title|                   w|forum_id|         forum_title| platform|           corpus_id|
>     +---------+--------------------+--------------------+--------+--------------------+---------+--------------------+
>     |  1005670| "Spara i madrassen"|Där,står,mycket,l...|      18|Ekonomi > Ekonomi...|flashback|flashback-ekonomi...|
>     |  1005847|Invester vinst in...|Om,du,sätter,av,t...|     258|Ekonomi > Företag...|flashback|flashback-ekonomi...|
>     |  1008130|Johan Norbergs En...|Tack,.,El,penal,d...|     293|Ekonomi > Nationa...|flashback|flashback-ekonomi...|
>     |  1008965|"Det stora raset"...|Om,tekniken,får,f...|     260|Ekonomi > Värdepa...|flashback|flashback-ekonomi...|
>     |  1011185|EF och dra av dyr...|På,företaget,där,...|     258|Ekonomi > Företag...|flashback|flashback-ekonomi...|
>     |  1013160|    Vilken Kinafond?|Jag,tror,på,HQ,Ki...|     260|Ekonomi > Värdepa...|flashback|flashback-ekonomi...|
>     |  1014006|Förvara kontanter...|Det,är,så,extrema...|     294|Ekonomi > Private...|flashback|flashback-ekonomi...|
>     |  1015049|     Inköp i företag|Är,ny,inom,företa...|     258|Ekonomi > Företag...|flashback|flashback-ekonomi...|
>     |   101813|     Startkapitalet?|Av,MVH,/,/,Detta,...|      18|Ekonomi > Ekonomi...|flashback|flashback-ekonomi...|
>     |  1025347|           Blancålån|hjälp,!,Sen,tycke...|     294|Ekonomi > Private...|flashback|flashback-ekonomi...|
>     |  1032576|           korvkiosk|Finns,vanliga,kor...|     258|Ekonomi > Företag...|flashback|flashback-ekonomi...|
>     |  1034516|Vilka Kina fonder...|Vill,man,höja,ris...|     489|    Ekonomi > Fonder|flashback|flashback-ekonomi...|
>     |  1036269|Handla optioner i...|Visst,stämmer,det...|     260|Ekonomi > Värdepa...|flashback|flashback-ekonomi...|
>     |  1037036|Vad tjänar en för...|Jag,jobbar,också,...|      18|Ekonomi > Ekonomi...|flashback|flashback-ekonomi...|
>     |  1039302|Kapitalförsäkring...|Brytpunkten,för,d...|     260|Ekonomi > Värdepa...|flashback|flashback-ekonomi...|
>     |  1041102|    Begreppet pengar|Statligt,utfärdad...|     293|Ekonomi > Nationa...|flashback|flashback-ekonomi...|
>     |  1041767|      Avräkningsnota|första,alternativ...|     260|Ekonomi > Värdepa...|flashback|flashback-ekonomi...|
>     |  1042680|Utländska indexfo...|Tycker,själv,att,...|     489|    Ekonomi > Fonder|flashback|flashback-ekonomi...|
>     |  1043999|Tulldeklaration, ...|så,vad,mer,behöve...|     258|Ekonomi > Företag...|flashback|flashback-ekonomi...|
>     |  1045539|        mini futures|handlar,med,RBS,....|     260|Ekonomi > Värdepa...|flashback|flashback-ekonomi...|
>     +---------+--------------------+--------------------+--------+--------------------+---------+--------------------+
>     only showing top 20 rows

In [ ]:
// Overview of number of threads in different subforums

df_FL.groupBy($"forum_title").agg(count($"forum_title").as("count")).show(false)

  

>     +----------------------------------------------------------+-----+
>     |forum_title                                               |count|
>     +----------------------------------------------------------+-----+
>     |Allmänna rubriker > Ekonomi & juridik > Konsument/Inhandla|26307|
>     |Allmänna rubriker > Ekonomi & juridik > Familjerätt       |1623 |
>     |Allmänna rubriker > Ekonomi & juridik > Övrigt            |1638 |
>     |Allmänna rubriker > Ekonomi & juridik > Ekonomi           |24284|
>     |Allmänna rubriker > Ekonomi & juridik > Spartips          |217  |
>     |Allmänna rubriker > Ekonomi & juridik > Juridik           |3246 |
>     |Allmänna rubriker > Ekonomi & juridik > Företagande       |957  |
>     |Allmänna rubriker > Ekonomi & juridik > Lån & skulder     |1168 |
>     +----------------------------------------------------------+-----+

In [ ]:
df_FB.groupBy($"forum_title").agg(count($"forum_title").as("count")).show(false)

  

>     +---------------------------------------------------+-----+
>     |forum_title                                        |count|
>     +---------------------------------------------------+-----+
>     |Ekonomi > Privatekonomi                            |15154|
>     |Ekonomi > Bitcoin och andra virtuella valutor      |524  |
>     |Ekonomi > Värdepapper, valutor och råvaror: allmänt|8300 |
>     |Ekonomi > Fonder                                   |645  |
>     |Ekonomi > Företagande och företagsekonomi          |12139|
>     |Ekonomi > Nationalekonomi                          |3114 |
>     |Ekonomi > Aktier                                   |508  |
>     |Ekonomi > Ekonomi: övrigt                          |19308|
>     |Ekonomi > Offshore och skatteplanering             |1567 |
>     +---------------------------------------------------+-----+

In [ ]:
// Set parameters, and perform two different LDAs, one on FB and one on FL

val min_token = 4 // only accept tokens larger or equal to
val vocabSize = 10000 // Size of the vocab dictionary
val minDF = 5 // The minimum number of different documents a term must appear in to be included in the vocabulary
val minTF = 2 // The minimum number of times a term has to appear in a document to be included in the vocabulary

val numTopics_FL = no_forums(df_FL).toInt //4 // Number of topics in LDA model
val numTopics_FB = no_forums(df_FB).toInt
val maxIter = 8  // Maximum number of iterations for LDA model
val stopwords = stopwordList
val (ldaModel_FL, vectorizer_FL) = get_lda(df_FL, min_token, vocabSize, minDF, numTopics_FL, maxIter, stopwords)
val (ldaModel_FB, vectorizer_FB) = get_lda(df_FB, min_token, vocabSize, minDF, numTopics_FB, maxIter, stopwords)

  

>     +----------------------------------------------------------+
>     |forum_title                                               |
>     +----------------------------------------------------------+
>     |Allmänna rubriker > Ekonomi & juridik > Konsument/Inhandla|
>     |Allmänna rubriker > Ekonomi & juridik > Familjerätt       |
>     |Allmänna rubriker > Ekonomi & juridik > Övrigt            |
>     |Allmänna rubriker > Ekonomi & juridik > Ekonomi           |
>     |Allmänna rubriker > Ekonomi & juridik > Spartips          |
>     |Allmänna rubriker > Ekonomi & juridik > Juridik           |
>     |Allmänna rubriker > Ekonomi & juridik > Företagande       |
>     |Allmänna rubriker > Ekonomi & juridik > Lån & skulder     |
>     +----------------------------------------------------------+
>
>     +---------------------------------------------------+
>     |forum_title                                        |
>     +---------------------------------------------------+
>     |Ekonomi > Privatekonomi                            |
>     |Ekonomi > Bitcoin och andra virtuella valutor      |
>     |Ekonomi > Värdepapper, valutor och råvaror: allmänt|
>     |Ekonomi > Fonder                                   |
>     |Ekonomi > Företagande och företagsekonomi          |
>     |Ekonomi > Nationalekonomi                          |
>     |Ekonomi > Aktier                                   |
>     |Ekonomi > Ekonomi: övrigt                          |
>     |Ekonomi > Offshore och skatteplanering             |
>     +---------------------------------------------------+
>
>     min_token: Int = 4
>     vocabSize: Int = 10000
>     minDF: Int = 5
>     minTF: Int = 2
>     numTopics_FL: Int = 8
>     numTopics_FB: Int = 9
>     maxIter: Int = 8
>     stopwords: Array[String] = Array(bara, lite, finns, vill, samt, inga, även, finns, ganska, också, igen, just, that, with, http, jpg, kanske, tycker, gillar, bra, the, 000, måste, tjej, tjejer, tjejen, tjejerna, kvinna, kvinnor, kille, killar, killen, män, rätt, män, com, and, html, många, aldrig, www, mpg, avi, wmv, fan, förhelvetet, riktigt, känner, väldigt, font, size, mms, 2008, 2009, 95kr, dom, får, ska, kommer, två, vet, mer, pengar, pengarna, göra, fick, tror, andra, helt, kunna, behöver, betala, inget, dock, inget, tack, word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre, och, det, att, i, en, jag, hon, som, han, på, den, med, var, sig, för, så, till, är, men, ett, om, hade, de, av, icke, mig, du, henne, då, sin, nu, har, inte, hans, honom, skulle, hennes, där, min, man, ej, vid, kunde, något, från, ut, när, efter, upp, vi, dem, vara, vad, över, än, dig, kan, sina, här, ha, mot, alla, under, någon, eller, allt, mycket, sedan, ju, denna, själv, detta, åt, utan, varit, hur, ingen, mitt, ni, bli, blev, oss, din, dessa, några, deras, blir, mina, samma, vilken, er, sådan, vår, blivit, dess, inom, mellan, sådant, varför, varje, vilka, ditt, vem, vilket, sitta, sådana, vart, dina, vars, vårt, våra, ert, era, vilkas)
>     ldaModel_FL: org.apache.spark.mllib.clustering.LDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@10a74a7d
>     vectorizer_FL: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_92cbca9d69d3
>     ldaModel_FB: org.apache.spark.mllib.clustering.LDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@28741094
>     vectorizer_FB: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_aaaff6faf1cc

  

### Results

Here we will visualize the most important part of the different topic
distributions, both for FB and for FL.

In [ ]:
// Here we pick out the most important terms in each topic for FL.

val topicIndices = ldaModel_FL.describeTopics(maxTermsPerTopic = 5)
val vocabList = vectorizer_FL.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics_FL topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     8 topics:
>     TOPIC 0
>     tillbaka	0.008216191335042956
>     skatt	0.006509257902643634
>     betalar	0.005635041121233399
>     folk	0.005131491885839656
>     jobb	0.0051057124094540714
>     ==========
>     TOPIC 1
>     barn	0.013917573767387432
>     spara	0.00891926717912398
>     skrev	0.008795787566591128
>     följande	0.007687478743540771
>     köpa	0.007421083954132069
>     ==========
>     TOPIC 2
>     jobb	0.005941548013271099
>     banken	0.005184644368971845
>     betalar	0.004761136982814706
>     jobbar	0.004732367048439004
>     tillbaka	0.00447708103221914
>     ==========
>     TOPIC 3
>     köper	0.011086173062869036
>     handlar	0.010707762120455214
>     äter	0.007974933506441925
>     köpa	0.007856980307486902
>     vuxna	0.005976512509098039
>     ==========
>     TOPIC 4
>     köpa	0.009345022745034834
>     kläder	0.007752851561346635
>     köpte	0.0057978627025462625
>     barn	0.005201306667796932
>     kolla	0.004548491158766831
>     ==========
>     TOPIC 5
>     barn	0.02047574524007617
>     barnet	0.009776681578582441
>     barnen	0.005742962450413186
>     skrev	0.004847780768717609
>     jobb	0.004608856784822973
>     ==========
>     TOPIC 6
>     betalar	0.010248488991996553
>     ekonomi	0.008863292049078849
>     huset	0.0068847763666443585
>     köpa	0.006869607591782869
>     barn	0.0062769843242390586
>     ==========
>     TOPIC 7
>     buffert	0.022462688377360988
>     kontantinsats	0.01832413857067284
>     000kr	0.012844781601955733
>     resa	0.008253097762916517
>     amortering	0.005493083255139651
>     ==========
>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(8, 51, 3, 21, 7),Array(0.008216191335042956, 0.006509257902643634, 0.005635041121233399, 0.005131491885839656, 0.0051057124094540714)), (Array(0, 15, 2, 11, 1),Array(0.013917573767387432, 0.00891926717912398, 0.008795787566591128, 0.007687478743540771, 0.007421083954132069)), (Array(7, 52, 3, 24, 8),Array(0.005941548013271099, 0.005184644368971845, 0.004761136982814706, 0.004732367048439004, 0.00447708103221914)), (Array(5, 12, 168, 1, 161),Array(0.011086173062869036, 0.010707762120455214, 0.007974933506441925, 0.007856980307486902, 0.005976512509098039)), (Array(1, 6, 19, 0, 16),Array(0.009345022745034834, 0.007752851561346635, 0.0057978627025462625, 0.005201306667796932, 0.004548491158766831)), (Array(0, 40, 27, 2, 7),Array(0.02047574524007617, 0.009776681578582441, 0.005742962450413186, 0.004847780768717609, 0.004608856784822973)), (Array(3, 30, 37, 1, 0),Array(0.010248488991996553, 0.008863292049078849, 0.0068847763666443585, 0.006869607591782869, 0.0062769843242390586)), (Array(157, 370, 166, 226, 469),Array(0.022462688377360988, 0.01832413857067284, 0.012844781601955733, 0.008253097762916517, 0.005493083255139651)))
>     vocabList: Array[String] = Array(barn, köpa, skrev, betalar, alltid, köper, kläder, jobb, tillbaka, saker, olika, följande, handlar, kostar, brukar, spara, kolla, bättre, svar, köpte, borde, folk, hemma, lika, jobbar, faktiskt, precis, barnen, fråga, istället, ekonomi, annars, runt, mindre, inkomst, sambo, hoppas, huset, känns, köpt, barnet, bort, ihop, verkar, alls, håller, problem, dessutom, stor, månad, svårt, skatt, banken, förstår, tjänar, låter, tips, extra, direkt, jobba, länge, räkningar, mamma, klart, större, tyvärr, beror, konto, handla, vissa, ofta, massa, tänker, sånt, liten, ränta, sparar, egentligen, undrar, ville, lägger, skriva, däremot, stora, sälja, billigare, veta, mest, själva, säkert, absolut, tänkte, tänka, enligt, leva, nästan, bilen, föräldrar, först, hitta, sett, funkar, hyra, enkelt, använda, självklart, låna, enda, fortfarande, tidigare, sambon, ungefär, använder, börja, egen, egna, längre, nytt, lätt, utgifter, ringa, kort, lägenhet, börjar, behöva, sitter, summa, billigt, hittar, trots, tillsammans, räkna, pappa, tråden, bästa, liknande, företag, skicka, bank, dyrt, jobbet, flytta, räknar, klarar, pris, hjälpa, lever, säljer, slut, vilja, högre, betalt, anser, lånet, okej, ifrån, oftast, buffert, människor, gott, hålla, vuxna, koll, sätta, tagit, klara, 000kr, dyrare, äter, byta, betalat, jobbat, stort, välja, dotter, hand, tänkt, person, väljer, lycka, gratis, familj, bäst, delar, eget, oavsett, nätet, boende, gemensam, långt, frågan, sidan, gamla, dåligt, sparande, bidrag, tyckte, minst, vanliga, räcker, händer, haha, konstigt, vänta, plus, sonen, priset, hört, undan, sida, summan, drar, dela, betalade, kostade, månaden, räknas, skaffa, skillnad, värt, möjligt, skrivit, exakt, försöka, månader, ersättning, resa, fungerar, försöker, kostnader, läsa, endast, papper, trodde, räntan, hemsida, sätter, inlägg, lägenheten, beroende, prata, skulder, spelar, vanligt, tänk, resten, innebär, liksom, speciellt, massor, söka, gemensamma, skor, säljaren, pappan, dyra, vore, lämna, emot, nöjd, tusen, halva, livet, kontot, roll, ingenting, hittade, åker, höra, början, funderar, butik, namn, igenom, svara, tråd, lönen, enbart, ring, gemensamt, läst, familjen, kassan, heltid, visa, kassa, fattar, personer, arbetslös, fina, ansvar, totalt, ekonomiskt, ringer, paket, ringde, började, nått, slipper, dottern, barnbidraget, satt, telefon, inkomster, såklart, exempel, betalas, internet, företaget, storlek, dålig, vatten, dagis, miljoner, passar, älskar, stämmer, ingår, personen, tanke, vänner, hälften, arbete, skönt, lagt, äger, utbildning, jättebra, varor, lösa, stod, sparat, tjäna, kosta, behövs, låta, försäkring, vagn, fort, ekonomin, frakt, viktigt, möjlighet, kompis, snabbt, glad, sönder, problemet, skit, övrigt, mamman, plats, tydligen, gånger, reda, lägre, situation, vanlig, varan, aning, förstå, semester, bilar, maken, säker, hyran, varandra, tacksam, erfarenhet, billiga, kontantinsats, butiken, skolan, särskilt, kontakt, iallafall, rent, skickar, såna, tycka, mesta, betyder, viss, alternativ, affären, avtal, lilla, 3000, mail, köra, bostad, hittat, kollar, handlat, fint, visste, läser, fonder, förutom, ifall, sparkonto, stan, blöjor, valt, kontakta, givetvis, litet, alldeles, pratar, tiden, maten, priser, väntar, ensam, 2000, behov, antar, knappt, anledning, kollat, läste, roligt, jobbade, sjuk, 5000, skatten, söker, fyller, känna, gifta, japp, räknat, våran, börjat, avgift, posten, slippa, barnbidrag, 1000, vagnen, titta, jätte, affär, beställa, intressant, frågade, iväg, nyfiken, saken, betydligt, förklara, flyttade, antingen, fixa, sällan, bostadsrätt, föräldrarna, kräva, svenska, hamnar, nära, kollade, dras, hjälper, synd, billig, kostnaden, värde, flyttar, amortering, berätta, försäkringar, gången, frågor, rabatt, knappast, någonstans, brev, sjukt, sluta, bensin, sålt, ekonomiska, antal, mannen, räkningarna, illa, svart, blocket, kreditkort, gammal, hända, därifrån, sparade, minns, jobbigt, underhåll, skickat, medan, frågar, resor, fullt, arbetar, otroligt, sämre, tvungen, julklappar, orkar, snälla, landet, senare, föräldrapenning, mobil, kostnad, plagg, gravid, regler, amortera, iofs, kläderna, krav, kortet, princip, inkl, bodde, rimligt, överens, 4000, kronofogden, lättare, höga, kredit, vinst, lång, behövde, form, skull, affärer, konton, möbler, menade, bostadsbidrag, normalt, förstås, föräldraledig, hehe, barnets, prova, förut, banker, sjukskriven, privat, handlade, plugga, fortsätta, kött, kontant, ihåg, räkning, dels, visar, säljare, byxor, körkort, butiker, hänt, lånen, kring, vecka, vårdnad, kräver, naturligtvis, onödigt, förälder, övriga, svaret, mjölk, budget, bild, barnens, sålde, skickade, köpet, lära, bröd, tråkigt, förmodligen, behålla, vägen, himla, nämligen, arbeta, sant, avdrag, rosa, använt, köparen, ändra, tröja, tilläggskommentar, skuld, inköp, valde, info, trådstartarens, fasta, coop, beslut, förslag, nordea, utöver, pass, testa, beställt, facket, personligen, äldre, faktura, belopp, ställe, räntor, intresserad, utomlands, nöjen, antagligen, full, fundera, paketet, arbetsgivare, hämta, avbetalning, lånar, tillräckligt, polisen, jeans, fylla, räcka, fallet, kunder, snarare, förrän, priserna, märke, dator, åring, lunch, såhär, lidl, studielån, perfekt, löser, ensamstående, ekonomisk, kommun, ställa, önskar, bröllop, lämnar, bygga, 1500, krävs, bebis, kronor, starta, forum, försörja, beställde, uppgifter, därmed, 2007, villa, information, stället, chans, världen, delad, försökt, ställen, dumt, leta, stad, begagnat, lyckas, bilder, vuxen, aktier, löner, pratat, bott, högt, bryr, studerar, svarar, skillnaden, förstått, gekås, jacka, glömde, grund, tillägga, tala, godis, nöjda, märken, tjänst, tanken, borta, avgifter, shoppa, syster, hinner, laga, senaste, tillgångar, skulden, öppna, plötsligt, situationen, skriv, ovan, tittar, krona, största, automatiskt, smart, hamna, miljon, beloppet, snygga, anmäla, gett, 6000, leksaker, pluggar, ytterligare, omöjligt, kommunen, lagen, utav, sparkontot, pratade, drygt, människa, vända, puffar, visserligen, sättet, förresten, bestämma, bestämt, datum, barnkläder, frakten, bolag, skatteverket, present, hyresrätt, studier, jävla, kvitto, förhoppningsvis, bankerna, skyldig, ordentligt, sambos, produkter, dags, annorlunda, arbetsgivaren, tillkommer, ringt, jämföra, amorterar, rädd, fattig, passa, samhället, moms, erbjudande, säkerhet, ålder, risk, summor, middag, trist, exempelvis, försök, inkomsten, puff, svarade, jurist, betalda, vårat, dagens, vägrar, värd, allra, staten, minsta, värsta, låga, förhållande, bolån, hänger, dåliga, böcker, kund, trådar, telefonen, räkningen, kostnaderna, anledningen, försvinner, huvudet, usch, hemifrån, håll, lagar, timmar, student, lägsta, utifrån, skiljer, bredband, grej, klar, julklapp, vare, planerar, begära, jaha, flyttat, letar, långa, guld, tillgång, noga, angående, tjänat, ledig, sidor, färg, tillbaks, hårt, planera, advokat, bostaden, igång, klänning, studera, hund, skäl, poäng, extremt, kvalitet, används, systemet, socialbidrag, ställer, isåfall, nuvarande, kompisar, röra, reklam, definitivt, syskon, enskild, ställt, utmärkt, relativt, pension, snygg, datorn, suck, träffa, tvärtom, åkte, öppet, vågar, missat, risken, vinsten, frukt, minus, land, trött, mammaledig, service, spel, chef, svensk, gymnasiet, värdet, funka, avgiften, riktig, snål, anställd, läget, försäljning, säng, osäker, gift, överallt, diskutera, velat, förlorar, bevisa, fakturan, främst, djur, sådär, bestämmer, överhuvudtaget, grönsaker, modell, anställning, kalla, strl, lagom, förhand, åsikter, prylar, länken, skola, följa, 7000, adress, trivs, köpare, kunden, boendet, goda, super, saknar, kram, sköter, skaffar, medans, presenter, tackar, lägg, vitt, sakerna, lånade, kände, byter, medlem, mäklaren, slags, jämför, därefter, tipset, föräldern, täcker, huslån, lånat, billigaste, väll, åsikt, tillfället, slutar, kaffe, 500kr, ansöka, unna, uppe, storlekar, testamente, grattis, rätten, sorry, 8000, tipsa, spar, utom, barns, räknade, provat, hemmet, kanon, troligen, användas, nivå, privata, lyckats, deklarationen, område, senast, plocka, renovering, 2500, toppen, separat, sökt, roliga, slänga, sambons, spendera, egendom)
>     topics: Array[Array[(String, Double)]] = Array(Array((tillbaka,0.008216191335042956), (skatt,0.006509257902643634), (betalar,0.005635041121233399), (folk,0.005131491885839656), (jobb,0.0051057124094540714)), Array((barn,0.013917573767387432), (spara,0.00891926717912398), (skrev,0.008795787566591128), (följande,0.007687478743540771), (köpa,0.007421083954132069)), Array((jobb,0.005941548013271099), (banken,0.005184644368971845), (betalar,0.004761136982814706), (jobbar,0.004732367048439004), (tillbaka,0.00447708103221914)), Array((köper,0.011086173062869036), (handlar,0.010707762120455214), (äter,0.007974933506441925), (köpa,0.007856980307486902), (vuxna,0.005976512509098039)), Array((köpa,0.009345022745034834), (kläder,0.007752851561346635), (köpte,0.0057978627025462625), (barn,0.005201306667796932), (kolla,0.004548491158766831)), Array((barn,0.02047574524007617), (barnet,0.009776681578582441), (barnen,0.005742962450413186), (skrev,0.004847780768717609), (jobb,0.004608856784822973)), Array((betalar,0.010248488991996553), (ekonomi,0.008863292049078849), (huset,0.0068847763666443585), (köpa,0.006869607591782869), (barn,0.0062769843242390586)), Array((buffert,0.022462688377360988), (kontantinsats,0.01832413857067284), (000kr,0.012844781601955733), (resa,0.008253097762916517), (amortering,0.005493083255139651)))

In [ ]:
// Here we pick out the most important terms in each topic for FB.

val topicIndices = ldaModel_FB.describeTopics(maxTermsPerTopic = 5)
val vocabList = vectorizer_FB.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics_FB topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     9 topics:
>     TOPIC 0
>     aktier	0.010372609709935948
>     fonder	0.009985682032437188
>     köpa	0.007147380184535501
>     företag	0.004942859552056198
>     olika	0.004763000081202069
>     ==========
>     TOPIC 1
>     skatt	0.014364304931778378
>     företag	0.008645452621562126
>     företaget	0.008300389229465058
>     bolaget	0.007903692818210637
>     bolag	0.006402673712961038
>     ==========
>     TOPIC 2
>     folk	0.0052167273044414555
>     jobb	0.004475724890639614
>     köpa	0.0044337052877254195
>     bättre	0.003773904791916776
>     istället	0.003169961913268716
>     ==========
>     TOPIC 3
>     bitcoin	0.015097168905144656
>     bitcoins	0.009816539841970279
>     köpa	0.0040680120997631155
>     företag	0.0032437136384805445
>     olika	0.0030445615311591244
>     ==========
>     TOPIC 4
>     moms	0.00633827140783334
>     köpa	0.005959993345718559
>     företag	0.005906188488700269
>     sälja	0.005238515382692445
>     betalar	0.004281708500936429
>     ==========
>     TOPIC 5
>     banken	0.00864432335805033
>     konto	0.008339277536899259
>     bank	0.007340445595021358
>     tillbaka	0.005965709762899769
>     ränta	0.005342093657300021
>     ==========
>     TOPIC 6
>     köpa	0.0034366699860672793
>     banken	0.003289013342902767
>     moms	0.003103723942186164
>     köper	0.003101990581430088
>     will	0.0030185287141540774
>     ==========
>     TOPIC 7
>     inflation	0.007176851887240197
>     guld	0.006395635479842193
>     staten	0.006005858645610903
>     valuta	0.005155325598855097
>     värde	0.0049879413505575666
>     ==========
>     TOPIC 8
>     aktier	0.012040212950249483
>     köpa	0.007639727547960423
>     aktien	0.006203784300724798
>     bolag	0.0057716967783410945
>     aktie	0.004584515168344491
>     ==========
>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(1, 42, 0, 2, 15),Array(0.010372609709935948, 0.009985682032437188, 0.007147380184535501, 0.004942859552056198, 0.004763000081202069)), (Array(7, 2, 32, 40, 23),Array(0.014364304931778378, 0.008645452621562126, 0.008300389229465058, 0.007903692818210637, 0.006402673712961038)), (Array(3, 25, 0, 4, 9),Array(0.0052167273044414555, 0.004475724890639614, 0.0044337052877254195, 0.003773904791916776, 0.003169961913268716)), (Array(554, 726, 0, 2, 15),Array(0.015097168905144656, 0.009816539841970279, 0.0040680120997631155, 0.0032437136384805445, 0.0030445615311591244)), (Array(115, 0, 2, 6, 10),Array(0.00633827140783334, 0.005959993345718559, 0.005906188488700269, 0.005238515382692445, 0.004281708500936429)), (Array(5, 22, 20, 12, 13),Array(0.00864432335805033, 0.008339277536899259, 0.007340445595021358, 0.005965709762899769, 0.005342093657300021)), (Array(0, 5, 115, 16, 993),Array(0.0034366699860672793, 0.003289013342902767, 0.003103723942186164, 0.003101990581430088, 0.0030185287141540774)), (Array(220, 87, 65, 254, 56),Array(0.007176851887240197, 0.006395635479842193, 0.006005858645610903, 0.005155325598855097, 0.0049879413505575666)), (Array(1, 0, 63, 23, 100),Array(0.012040212950249483, 0.007639727547960423, 0.006203784300724798, 0.0057716967783410945, 0.004584515168344491)))
>     vocabList: Array[String] = Array(köpa, aktier, företag, folk, bättre, banken, sälja, skatt, borde, istället, betalar, verkar, tillbaka, ränta, fråga, olika, köper, mindre, alltid, stor, bank, problem, konto, bolag, större, jobb, stora, direkt, runt, lika, svårt, svar, företaget, precis, känns, handlar, saker, förstår, dessutom, kort, bolaget, enkelt, fonder, enligt, låna, högre, säljer, håller, vinst, brukar, kostar, kolla, annars, faktiskt, låter, svenska, värde, egentligen, spara, börja, bort, tidigare, tjänar, aktien, länge, staten, marknaden, enda, säkert, ekonomi, fortfarande, använda, liknande, inkomst, mest, räntan, frågan, tänker, hoppas, vissa, börsen, ihop, börjar, tjäna, tips, klart, banker, guld, däremot, sett, fungerar, miljoner, intressant, exempel, sitter, tråden, kapital, hålla, människor, egen, aktie, möjligt, alls, tänkte, massa, avkastning, jobbar, liten, ifrån, först, beror, köpte, risk, endast, hitta, moms, undrar, bästa, använder, tänka, investera, sätta, innebär, själva, ofta, jobba, stort, jävla, längre, skulder, räkna, handla, priset, leva, bankerna, visa, skriva, egna, trots, lägre, kursen, person, problemet, kontot, hyra, veta, köpt, extra, händer, starta, pris, eget, äger, nästan, månad, sidan, vore, självklart, ungefär, värt, lätt, summa, länder, läsa, tänkt, koll, tyvärr, skicka, givetvis, lånet, land, försöka, skrev, gratis, flytta, behöva, världen, ifall, billigare, vilja, absolut, exakt, form, alternativ, tanke, kostnader, värdet, nytt, avgifter, lägenhet, ökar, lägger, funkar, emot, försöker, långt, minst, enbart, snabbt, igenom, information, stämmer, personer, oavsett, vanligt, vänta, spelar, skit, inlägg, kunder, speciellt, skatten, hemma, tråd, tillgångar, bäst, drar, hittar, betalt, räknar, inflation, ekonomin, totalt, skaffa, plus, barn, beroende, antar, anser, hjälpa, dåligt, största, lever, viss, frågor, risken, sida, roll, sånt, avgift, dagens, lånar, tagit, väljer, gott, vinsten, välja, exempelvis, princip, säker, arbete, kontanter, början, förklara, valuta, varor, antal, knappast, övrigt, läst, rent, ingenting, landet, okej, betyder, ville, utgifter, belopp, bidrag, snarare, tänk, visar, utdelning, oftast, sparande, skatteverket, namn, kring, fond, slut, sätter, därmed, index, lyckas, system, 000kr, anledning, förmodligen, skillnad, skapa, såklart, gamla, skuld, delar, räcker, bilen, firma, möjlighet, nordea, köra, resten, livet, sparar, låta, vanliga, betydligt, sker, billigt, hand, betalat, relativt, hört, produkter, krävs, klarar, lycka, undan, slipper, svensk, lång, fortsätta, bostad, ytterligare, jobbet, hemsida, tusen, sålde, summan, byta, satsa, senare, internet, öppna, särskilt, förstå, ekonomiska, avtal, sälj, resultat, svara, betalas, verksamhet, funderar, fallet, sålt, kortet, kräver, tjänster, priserna, antingen, dålig, klara, tiden, utomlands, höga, lösa, antagligen, privat, pratar, ekonomiskt, priser, dyrare, förutom, personen, investering, kund, reda, papper, satt, medan, avdrag, skickar, förlust, senaste, haha, erfarenhet, fort, situation, aktierna, naturligtvis, personligen, extremt, ringa, kronofogden, siffror, tillräckligt, behövs, trodde, summor, låga, bankkonto, föräldrar, dyrt, räknas, omöjligt, sluta, inflationen, systemet, intresserad, investeringar, grund, försäljning, skulden, lönen, zero, började, vanlig, kollar, svaret, hända, enskild, kronor, tillgång, ovan, dela, tydligen, övriga, söka, sparkonto, sämre, kredit, info, kläder, kostnad, konstigt, boende, tacksam, konton, angående, anledningen, lära, privata, räntor, efterfrågan, huset, gånger, följande, bygga, säkerhet, tillväxt, kurs, affärer, läser, samhället, således, inkomster, igång, läste, miljarder, aning, marknad, skrivit, skatta, nuvarande, nivå, värd, lämna, någonstans, konkurs, ekonomisk, högt, börjat, fattar, skapar, krav, miljon, söker, förslag, främst, bilar, månader, slippa, förlora, lägenheten, förstått, sant, höra, innehav, hamnar, saken, jobbat, titta, tillsammans, faktura, flashback, helvete, skillnaden, nått, chans, stiger, fortsätter, frågar, fonden, tjänst, produkt, sikt, följer, skatter, talar, minskar, behålla, fixa, uppgifter, litet, läge, olja, utbildning, beloppet, vänner, långsiktigt, dels, vägen, uppåt, regler, nära, eventuellt, minus, momsen, fullt, framtida, ägare, dollar, normalt, lågt, pass, sättet, skiten, leder, förlorar, väntar, behov, kontakt, kostnaden, försök, kontakta, jävligt, viktigt, fungera, nätet, räkningar, resa, 1000, ökat, förhand, föra, åker, placera, håll, förstås, varandra, affär, tanken, driva, lagt, bitcoin, otroligt, plats, hyran, utav, följa, minska, utveckling, anställda, hälften, kompis, bostadsrätt, svart, kassa, tjänat, liksom, hårt, berätta, därefter, sjukt, arbeta, hittade, faller, kassan, vinster, analys, portfölj, amerikanska, påverkar, iväg, rejält, mesta, dras, stiga, sidor, prata, uppgång, knappt, brev, intresse, såhär, utöver, kosta, sjunker, kunden, räknat, världens, krona, spel, ställa, beslut, arbetar, dator, pension, positivt, totala, silver, cash, forum, hjälper, löner, motsvarande, rimligt, lättare, hittat, dollarn, tillfället, investerar, sparat, gammal, gången, uttag, lyckats, omsättning, framöver, alldeles, månaden, erbjuder, dyra, kontant, troligen, orkar, används, företagen, smart, procent, tittar, rika, handel, dåliga, google, plötsligt, steg, försvinner, betalning, courtage, fylla, generellt, kreditkort, insatt, spela, korrekt, stod, alternativt, plocka, nämligen, saknar, driver, minns, lägg, familj, betalade, arbetsgivare, bygger, skyldig, diverse, nöjd, ihåg, global, dags, billig, verksamheten, försäkring, hänt, medel, flyttar, kräva, dumt, riktiga, troligtvis, lösning, kraftigt, högst, ersättning, iallafall, sänka, slags, telefon, order, stället, läget, goda, avkastningen, tvungen, verkligheten, utdelningen, växa, inköp, lilla, fundera, konsumtion, överhuvudtaget, bostäder, svårare, massor, ställer, kunskap, rekommendera, gällande, nyheter, post, valutan, tvärtom, ändra, deklarationen, långa, förhoppningsvis, testa, kronan, vare, ring, kapitalet, bitcoins, allting, bryr, höja, fint, värdepapper, praktiken, borta, räntorna, framtiden, illa, desto, resultatet, poäng, tackar, undvika, säkra, anställd, investerare, tillbaks, sällan, sambo, mail, 2000, botten, vända, buffert, tyckte, användas, ansvar, kommande, tankar, valt, nere, billiga, halva, amortera, fastigheter, krediter, lånen, påverka, skiter, skäl, länk, visste, skolan, samband, kallas, väll, blocket, mynt, valutor, starkt, andel, kvitto, jämför, situationen, ämnet, spendera, ingår, lånat, adress, lämnar, kalla, fakturan, produktion, görs, kapitalförsäkring, automatiskt, gett, kollat, uppenbarligen, aktiebolag, enklare, roligt, projekt, privatperson, rapporten, resor, full, iofs, rapport, sedlar, möjligheter, svenskar, sjunka, nordnet, ringer, rekommenderar, slår, företagets, känna, politiker, försökt, varan, missat, faktum, källa, fina, leta, bolagen, växla, passar, menade, visserligen, tycka, uppe, löser, plan, äter, 5000, butik, jämföra, utländska, såna, diskutera, förut, påstår, produkten, skull, antalet, avgiften, samtliga, enskilda, tydligt, polare, hamna, argument, ökad, samhälle, värda, definitivt, eventuella, glad, någonsin, intäkter, folket, rätta, utvecklingen, värsta, fasta, osäker, anses, utifrån, intressanta, spännande, riktig, transaktioner, euro, vågar, garanterat, funderat, deklaration, risker, cirka, 100k, närmare, bolagets, back, framtid, arbetslös, resurser, vänder, startar, posten, säljaren, trådar, bild, huvudet, möjligheten, nedgång, fördel, ökade, sköter, handlas, växer, bolån, framförallt, firman, kollade, depå, slutar, budget, seriöst, tvingas, utveckla, gränsen, alternativet, synd, frågade, snackar, drygt, potential, program, fonderna, dylikt, butiker, nytta, köpet, netto, student, skönt, använt, ordentligt, förr, mamma, isåfall, total, böcker, kostnaderna, regel, senast, snabbare, stöd, tider, privatpersoner, stad, skickat, erbjuda, inser, förhållande, enkel, hämta, gissar, trycka, vettigt, mille, datum, säljare, amortering, obligationer, eventuell, bostaden, fattiga, tala, negativt, sannolikt, minsta, beskattas, paket, röra, ställe, ringde, stark, kunderna, vatten, förutsatt, slutet, nivåer, bestämmer, leda, överens, denne, portföljen, stabil, försäljningen, öppnar, säljs, utdelningar, reklam, internetbank, onödigt, investmentbolag, fördelar, this, skulderna, klar, will, oberoende, svarta, bevisa, annorlunda, möjligen, bevis)
>     topics: Array[Array[(String, Double)]] = Array(Array((aktier,0.010372609709935948), (fonder,0.009985682032437188), (köpa,0.007147380184535501), (företag,0.004942859552056198), (olika,0.004763000081202069)), Array((skatt,0.014364304931778378), (företag,0.008645452621562126), (företaget,0.008300389229465058), (bolaget,0.007903692818210637), (bolag,0.006402673712961038)), Array((folk,0.0052167273044414555), (jobb,0.004475724890639614), (köpa,0.0044337052877254195), (bättre,0.003773904791916776), (istället,0.003169961913268716)), Array((bitcoin,0.015097168905144656), (bitcoins,0.009816539841970279), (köpa,0.0040680120997631155), (företag,0.0032437136384805445), (olika,0.0030445615311591244)), Array((moms,0.00633827140783334), (köpa,0.005959993345718559), (företag,0.005906188488700269), (sälja,0.005238515382692445), (betalar,0.004281708500936429)), Array((banken,0.00864432335805033), (konto,0.008339277536899259), (bank,0.007340445595021358), (tillbaka,0.005965709762899769), (ränta,0.005342093657300021)), Array((köpa,0.0034366699860672793), (banken,0.003289013342902767), (moms,0.003103723942186164), (köper,0.003101990581430088), (will,0.0030185287141540774)), Array((inflation,0.007176851887240197), (guld,0.006395635479842193), (staten,0.006005858645610903), (valuta,0.005155325598855097), (värde,0.0049879413505575666)), Array((aktier,0.012040212950249483), (köpa,0.007639727547960423), (aktien,0.006203784300724798), (bolag,0.0057716967783410945), (aktie,0.004584515168344491)))

In [ ]:
// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

// Create DF with proper column names
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

display(termDF)

  

[TABLE]

Truncated to 30 rows

In [ ]:
// Creates JSON data to display topic distribution of forum in FB
val rawJson = termDF.toJSON.collect().mkString(",\n")

In [ ]:
// Similar as above but for FL

val topicIndices = ldaModel_FL.describeTopics(maxTermsPerTopic = 6)
val vocabList = vectorizer_FL.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics_FL topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

// Create DF with proper column names
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

// Create JSON data to display topic distribution of forum in FB
val rawJson = termDF.toJSON.collect().mkString(",\n")

In [ ]:
def rawJson(lda_model: org.apache.spark.mllib.clustering.LDAModel, vectorizer: org.apache.spark.ml.feature.CountVectorizerModel): String = {
  val topicIndices = lda_model.describeTopics(maxTermsPerTopic = 6)
  val vocabList = vectorizer.vocabulary
  val topics = topicIndices.map { case (terms, termWeights) =>
    terms.map(vocabList(_)).zip(termWeights)
  }
  println(s"$numTopics_FL topics:")
  topics.zipWithIndex.foreach { case (topic, i) =>
    println(s"TOPIC $i")
    topic.foreach { case (term, weight) => println(s"$term\t$weight") }
    println(s"==========")
  }

  // Zip topic terms with topic IDs
  val termArray = topics.zipWithIndex

  // Transform data into the form (term, probability, topicId)
  val termRDD = sc.parallelize(termArray)
  val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
    val arrayOfTuple = x._1
    val topicId = x._2
    arrayOfTuple.map(el => (el._1, el._2, topicId))
  })

  // Create DF with proper column names
  val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

  // Create JSON data to display topic distribution
  termDF.toJSON.collect().mkString(",\n")
}
// Create JSON data to display topic distribution
val rawJson_FL = rawJson(ldaModel_FL, vectorizer_FL)
val rawJson_FB = rawJson(ldaModel_FB, vectorizer_FB)